In [1]:
!pip install regex tiktoken

Looking in indexes: https://pypi.ci.artifacts.walmart.com/artifactory/api/pypi/pythonhosted-pypi-release-remote/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 791.7/791.7 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.7 MB/s eta 0:00:001


In [2]:
# Exercise 1: Unicode Code Points
text = "Hello world 💩"
print(f"Original Text: {text}")

# Extract Unicode Code Points
code_points = [ord(char) for char in text]
print(f"Unicode Code Points: {code_points}")

Original Text: Hello world 💩
Unicode Code Points: [72, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100, 32, 128169]


In [3]:
# Exercise 2: UTF-8 Encoding
text = "Hello world 💩"
text_bytes = text.encode("utf-8")
byte_list = list(text_bytes)

print(f"Text String: {text}")
print(f"Byte List:   {byte_list}")
print(f"Length of String: {len(text)}")
print(f"Length of Bytes:  {len(byte_list)}")

Text String: Hello world 💩
Byte List:   [72, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100, 32, 240, 159, 146, 169]
Length of String: 13
Length of Bytes:  16


In [ ]:
# Implementation: BasicTokenizer
class BasicTokenizer:
    def __init__(self):
        self.merges = {} # (int, int) -> int
        self.vocab = {}  # int -> bytes

    def train(self, text, vocab_size, verbose=False):
        assert vocab_size >= 256
        num_merges = vocab_size - 256
        
        # Preprocessing: Convert string to UTF-8 bytes
        text_bytes = text.encode("utf-8")
        ids = list(text_bytes) # Integers 0-255
        
        for i in range(num_merges):
            stats = get_stats(ids)
            if not stats:
                break
            
            # Select the most frequent pair
            pair = max(stats, key=stats.get)
            
            # Mint a new token ID
            idx = 256 + i
            
            # Apply the merge
            ids = merge(ids, pair, idx)
            
            # Record the merge
            self.merges[pair] = idx
            
            if verbose:
                print(f"Merge {i+1}/{num_merges}: {pair} -> {idx} (Count: {stats[pair]})")
        
        # Build the final vocabulary map for decoding
        # 1. Initialize with raw bytes
        self.vocab = {idx: bytes([idx]) for idx in range(256)}
        # 2. Add merged tokens recursively
        for (p0, p1), idx in self.merges.items():
            self.vocab[idx] = self.vocab[p0] + self.vocab[p1]

    def decode(self, ids):
        # Concatenate bytes
        tokens = b"".join(self.vocab[idx] for idx in ids)
        # Decode to string, handling invalid byte sequences gracefully
        text = tokens.decode("utf-8", errors="replace")
        return text

    def encode(self, text):
        text_bytes = text.encode("utf-8")
        ids = list(text_bytes)
        
        while len(ids) >= 2:
            stats = get_stats(ids)
            # Find the pair in the current sequence with the lowest merge index
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            
            # If the best pair isn't in our learned merges, we are done
            if pair not in self.merges:
                break
            
            idx = self.merges[pair]
            ids = merge(ids, pair, idx)
        return ids

# Helper functions required by the class
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): 
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

SyntaxError: invalid syntax (2745100473.py, line 74)

In [5]:
# Exercise 3: Training the BasicTokenizer
tokenizer = BasicTokenizer()
text = "aaabdaaabac"
print(f"Input: {text}")

# Train for 3 merges (256 + 3 = 259 vocab size)
# We expect the most frequent pairs to be merged first.
tokenizer.train(text, vocab_size=259, verbose=True)

# Encode
encoded = tokenizer.encode(text)
print(f"Encoded: {encoded}")

# Decode
decoded = tokenizer.decode(encoded)
print(f"Decoded: {decoded}")

# Verify
assert text == decoded
print("Success: Decoded string matches input.")

NameError: name 'BasicTokenizer' is not defined

In [6]:
import regex as re # 'regex' module is required for \p{...} support

GPT2_SPLIT_PATTERN = r"""'(?:[sdmt]|ll|ve|re)|?\p{L}+|?\p{N}+|?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

In [7]:
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}|?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

In [ ]:
class RegexTokenizer(BasicTokenizer):
    def __init__(self, pattern=None):
        super().__init__()
        self.pattern = GPT4_SPLIT_PATTERN if pattern is None else pattern
        self.compiled_pattern = re.compile(self.pattern)

    def train(self, text, vocab_size, verbose=False):
        assert vocab_size >= 256
        num_merges = vocab_size - 256
        
        # 1. Split text into chunks using regex
        text_chunks = re.findall(self.compiled_pattern, text)
        
        # 2. Convert chunks to lists of byte integers
        ids = [list(ch.encode("utf-8")) for ch in text_chunks]
        
        for i in range(num_merges):
            stats = {}
            # Count pairs across ALL chunks
            for chunk_ids in ids:
                # Update stats dictionary in place
                for pair in zip(chunk_ids, chunk_ids[1:]):
                    stats[pair] = stats.get(pair, 0) + 1
            
            if not stats:
                break
            
            pair = max(stats, key=stats.get)
            idx = 256 + i
            
            # Apply merge to ALL chunks
            ids = [merge(chunk_ids, pair, idx) for chunk_ids in ids]
            
            self.merges[pair] = idx
            
        # Build vocabulary
        self.vocab = {idx: bytes([idx]) for idx in range(256)}
        for (p0, p1), idx in self.merges.items():
            self.vocab[idx] = self.vocab[p0] + self.vocab[p1]

    def encode(self, text):
        text_chunks = re.findall(self.compiled_pattern, text)
        ids = []
        for chunk in text_chunks:
            chunk_bytes = chunk.encode("utf-8")
            chunk_ids = list(chunk_bytes)
            # Apply BPE merges to this specific chunk
            chunk_ids = self._encode_chunk(chunk_ids)
            ids.extend(chunk_ids)
        return ids

    def _encode_chunk(self, ids):
        # Same encoding logic as BasicTokenizer, but for a single chunk
        while len(ids) >= 2:
            stats = get_stats(ids)
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            if pair not in self.merges:
                break
            idx = self.merges[pair]
            ids = merge(ids, pair, idx)
        return ids

SyntaxError: invalid syntax (1950933026.py, line 43)

In [9]:
# Exercise 4: Basic vs Regex Tokenizer
text = "Hello, world! 12345"
print(f"Test Text: {text}")

# Train Basic Tokenizer
print("--- Basic Tokenizer ---")
basic_enc = BasicTokenizer()
basic_enc.train(text, 260) # Train slightly
print("Encoded:", basic_enc.encode(text))

# Train Regex Tokenizer
print("--- Regex Tokenizer ---")
regex_enc = RegexTokenizer()
regex_enc.train(text, 260) # Train slightly
print("Encoded:", regex_enc.encode(text))

Test Text: Hello, world! 12345
--- Basic Tokenizer ---


NameError: name 'BasicTokenizer' is not defined

In [10]:
def bpe_reconstruct(mergeable_ranks, token, max_rank):
    parts = [bytes([b]) for b in token]
    while True:
        min_idx = None
        min_rank = None
        # Find the pair with the lowest rank (highest priority)
        for i, pair in enumerate(zip(parts[:-1], parts[1:])):
            rank = mergeable_ranks.get(pair + pair[1])
            if rank is not None and (min_rank is None or rank < min_rank):
                min_idx = i
                min_rank = rank
        if min_rank is None or (max_rank is not None and min_rank >= max_rank):
            break
        assert min_idx is not None
        # Merge
        parts = parts[:min_idx] + [parts[min_idx] + parts[min_idx + 1]] + parts[min_idx + 2:]
    return parts

def recover_merges(mergeable_ranks):
    merges = {}
    for token, rank in mergeable_ranks.items():
        if len(token) == 1:
            continue
        # Reconstruct how this token was built
        pair = tuple(bpe_reconstruct(mergeable_ranks, token, max_rank=rank))
        assert len(pair) == 2
        
        ix0 = mergeable_ranks[pair]
        ix1 = mergeable_ranks[pair[1]]
        merges[(ix0, ix1)] = rank
    return merges

In [11]:
# Exercise 5: Tiktoken and Special Tokens
import tiktoken

# Load GPT-4 tokenizer
enc = tiktoken.get_encoding("cl100k_base")

# Test encoding with special tokens
text = "Hello <|endoftext|> world"
try:
    # This will fail by default for safety
    print(enc.encode(text)) 
except Exception as e:
    print(f"Expected Error: {e}")

# Allow special tokens
tokens = enc.encode(text, allowed_special="all")
print(f"Tokens with special: {tokens}")

# Identify the ID for <|endoftext|>
eot_id = tokens[1]
print(f"ID for <|endoftext|>: {eot_id}")

Expected Error: Encountered text corresponding to disallowed special token '<|endoftext|>'.
If you want this text to be encoded as a special token, pass it to `allowed_special`, e.g. `allowed_special={'<|endoftext|>', ...}`.
If you want this text to be encoded as normal text, disable the check for this token by passing `disallowed_special=(enc.special_tokens_set - {'<|endoftext|>'})`.
To disable this check for all special tokens, pass `disallowed_special=()`.

Tokens with special: [9906, 220, 100257, 1917]
ID for <|endoftext|>: 220
